##  NFT (ERC-721) 數據分析

base on **`token_transfers`**

計算 https://etherscan.io/token/0x9e4ceA1ba3f1fdC80DD2b6CEb20628d03a1b1AEa

地址 `0x9e4ceA1ba3f1fdC80DD2b6CEb20628d03a1b1AEa` in ETH

- **当前持有人数**
- **总token数量**
- **holder的分布**

```sql
SELECT
  token_address,
  from_address,
  to_address,
  value as token_id,
  block_number,
  log_index
FROM `bigquery-public-data.crypto_ethereum.token_transfers`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) >= TIMESTAMP("2022-9-5")
  AND
  TIMESTAMP_TRUNC(block_timestamp, DAY) <= TIMESTAMP("2025-12-25")
  AND
  token_address = LOWER("0x9e4ceA1ba3f1fdC80DD2b6CEb20628d03a1b1AEa")
ORDER BY
  block_number ASC, -- 先排區塊 (大時間)
  log_index ASC     -- 再排區塊內的事件順序 (微觀時間)
```

In [9]:
import pandas as pd

df = pd.read_json("bquxjob_17275c16_19b51d8b284.json", lines=False)
df = df.sort_values(
    by=['block_number', 'log_index'],  # 指定排序欄位順序 (先排 block，再排 log)
    ascending=[True, True]             # 指定每個欄位是升序 (True) 還是降序 (False)
)
print(df.head())

                                token_address  \
0  0x9e4cea1ba3f1fdc80dd2b6ceb20628d03a1b1aea   
1  0x9e4cea1ba3f1fdc80dd2b6ceb20628d03a1b1aea   
2  0x9e4cea1ba3f1fdc80dd2b6ceb20628d03a1b1aea   
3  0x9e4cea1ba3f1fdc80dd2b6ceb20628d03a1b1aea   
4  0x9e4cea1ba3f1fdc80dd2b6ceb20628d03a1b1aea   

                                 from_address  \
0  0x0000000000000000000000000000000000000000   
1  0x0000000000000000000000000000000000000000   
2  0x0000000000000000000000000000000000000000   
3  0x0000000000000000000000000000000000000000   
4  0x0000000000000000000000000000000000000000   

                                   to_address  token_id  block_number  \
0  0xfcec606dd3baf197a520dd0dfa073343489b8049         1      15485610   
1  0xfcec606dd3baf197a520dd0dfa073343489b8049         2      15492197   
2  0xfcec606dd3baf197a520dd0dfa073343489b8049         3      15492197   
3  0xfcec606dd3baf197a520dd0dfa073343489b8049         4      15492197   
4  0xfcec606dd3baf197a520dd0dfa073343489b8049

In [25]:
token_df = df.groupby("token_id").last()
# print(token_df.head())
print(len(token_df.groupby("to_address"))) # holder
print(token_df.count()['to_address']) # token count 6000

holder_df = token_df.groupby("to_address").count()
holder_df = holder_df.sort_values("token_address")
print(holder_df.head(870)) # holder distribution

870
6000
                                            token_address  from_address  \
to_address                                                                
0xc479d3fd2f4a1a295fa190191446ff5b7d2c0551              1             1   
0xbc70ab3be4b0bffb56538943f7a6d7a8d25abb7e              1             1   
0xdab0306813667db2699466d149180dcdee0564e8              1             1   
0xaae159043315dad76ba431c970e6ee6ace1458aa              1             1   
0x985de337b99dd0a6fb1c4169d37a9e40cc5a2316              1             1   
...                                                   ...           ...   
0x282ae89028edfa715e85acdf91c28ae5a9afa94e             70            70   
0xfcec606dd3baf197a520dd0dfa073343489b8049            101           101   
0xa6da95e1583e4482f1e3325a9b49c083e312e117            105           105   
0xe252f34121e3e3d459a506e094e84fc62f8b7acc            175           175   
0x4bfac565e28a24c61a5ea1eefe78b6aede6efff2            250           250   

               